In [1]:
from kan import KAN
from collections import Counter
from sklearn.model_selection import KFold, cross_val_score, train_test_split
import matplotlib.pyplot as plt
from sklearn.datasets import make_moons
import torch
from sklearn.utils import shuffle
import random
import numpy as np
torch.set_default_dtype(torch.float32)

In [18]:
#读取数据
# sequences_test = []
# 
# # Open the text file in read mode
# with open('/tmp/pycharm_project_763/data/testsets/AAP_test.txt', 'r') as file:
#     lines = file.readlines()
# 
#     # Iterate through the lines to extract protein sequences
#     for line in lines:
#         if not line.startswith('>'):
#             # Strip any leading/trailing whitespace and add to the list
#             sequences_test.append(line.strip())
#             
# y_test = []
# 
# # Open the text file in read mode
# with open('/tmp/pycharm_project_763/data/testsets/label_AAP_test.txt', 'r') as file:
#     lines = file.readlines()
# 
#     # Iterate through each line
#     for line in lines:
#         # Split the line by whitespace and check if it has two columns
#         parts = line.strip().split()
#         if len(parts) == 2:
#             # The second part is the label
#             y_test.append(int(parts[1]))


In [19]:
#读取数据
# sequences = []
# 
# # Open the text file in read mode
# with open('/tmp/pycharm_project_763/data/trainsets/AAP.txt', 'r') as file:
#     lines = file.readlines()
# 
#     # Iterate through the lines to extract protein sequences
#     for line in lines:
#         if not line.startswith('>'):
#             # Strip any leading/trailing whitespace and add to the list
#             sequences.append(line.strip())
#             
# y = []
# 
# # Open the text file in read mode
# with open('/tmp/pycharm_project_763/data/trainsets/label_AAP_1.txt', 'r') as file:
#     lines = file.readlines()
# 
#     # Iterate through each line
#     for line in lines:
#         # Split the line by whitespace and check if it has two columns
#         parts = line.strip().split()
#         if len(parts) == 2:
#             # The second part is the label
#             y.append(int(parts[1]))


In [20]:
# len(sequences),len(sequences_test),len(y),len(y_test)

(214, 56, 214, 56)

In [35]:
def read_fasta(file_path):
    protein_list=[]
    label_list=[]
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()
            if line.startswith('>'):
                # 提取标签
                label = line.split('|')[1]
                label_list.append(int(label))
            else:
                # 提取蛋白质序列
                protein_list.append(line)

    return protein_list, label_list

# 示例文件路径
train_path = '/tmp/pycharm_project_763/raw/AVP.txt'
test_path ='/tmp/pycharm_project_763/raw/AVPT.txt'

sequences, y = read_fasta(train_path)
sequences_test, y_test = read_fasta(test_path)
len(sequences),len(sequences_test)

(951, 105)

In [36]:
def maccs_feature():
    maccs_list=[]
    with open('/tmp/pycharm_project_763/raw/maccs_AVP.txt','r') as f:
        for line in f:
            maccs=list(map(int,line.strip().split()))
            # print(maccs)
            maccs_list.append(maccs)
    return maccs_list
maccs_train=maccs_feature()
           
len(maccs_train),len(maccs_train[0])

(951, 167)

In [37]:
def maccs_feature():
    maccs_list=[]
    with open('/tmp/pycharm_project_763/raw/maccs_AVPT.txt','r') as f:
        for line in f:
            maccs=list(map(int,line.strip().split()))
            # print(maccs)
            maccs_list.append(maccs)
    return maccs_list
maccs_test=maccs_feature()
           
len(maccs_test),len(maccs_test[0])

(105, 167)

In [5]:
def calculate_aac(protein_list):
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

    # 初始化存储结果的列表
    aac_list = []

    for protein_sequence in protein_list:
        # 初始化AAC特征字典
        aac = {aa: 0 for aa in amino_acids}

        # 计算每种氨基酸在序列中的频率
        for aa in protein_sequence:
            if aa in aac:
                aac[aa] += 1

        # 将频率转换为比例
        sequence_length = len(protein_sequence)
        aac = [count / sequence_length for aa, count in aac.items()]

        # 将结果添加到列表中
        aac_list.append(aac)

    return aac_list

In [6]:
aac_train = calculate_aac(sequences)
aac_test = calculate_aac(sequences_test)
# aac_train.shape

In [7]:
NAKH900113 = {
    'A': 1.61, 'L': 1.37,
    'R': 0.40, 'K': 0.62,
    'N': 0.73, 'M': 1.59,
    'D': 0.75, 'F': 1.24,
    'C': 0.37, 'P': 0.67,
    'Q': 0.61, 'S': 0.68,
    'E': 1.50, 'T': 0.92,
    'G': 3.12, 'W': 1.63,
    'H': 0.46, 'Y': 0.67,
    'I': 1.61, 'V': 1.30
}
KRIW710101={
    'A': 4.60, 'L': 3.25,
    'R': 6.50, 'K': 7.90,
    'N': 5.90, 'M': 1.40,
    'D': 5.70, 'F': 3.20,
    'C': -1.00, 'P': 7.00,
    'Q': 6.10, 'S': 5.25,
    'E': 5.60, 'T': 4.80,
    'G': 7.60, 'W': 4.00,
    'H': 4.50, 'Y': 4.35,
    'I': 2.60, 'V': 3.40
}
HUTJ700103={
    'A': 154.33, 'L': 232.30,
    'R': 341.01, 'K': 300.46,
    'N': 207.90, 'M': 202.65,
    'D': 194.91, 'F': 204.74,
    'C': 219.79, 'P': 179.93,
    'Q': 235.51, 'S': 174.06,
    'E': 223.16, 'T': 205.80,
    'G': 127.90, 'W': 237.01,
    'H': 242.54, 'Y': 229.15,
    'I': 233.21, 'V': 207.60
}
ZIMJ680103={
    'A': 0.00, 'L': 0.13,
    'R': 52.00, 'K': 49.50,
    'N': 3.38, 'M': 1.43,
    'D': 49.70, 'F': 0.35,
    'C': 1.48, 'P': 1.58,
    'Q': 3.53, 'S': 1.67,
    'E': 49.90, 'T': 1.66,
    'G': 0.00, 'W': 2.10,
    'H': 51.60, 'Y': 1.61,
    'I': 0.13, 'V': 0.13
}
TANS770104= {
    'A': 1.194, 'L': 0.595,
    'R': 0.795, 'K': 1.060,
    'N': 0.659, 'M': 0.831,
    'D': 1.056, 'F': 0.377,
    'C': 0.678, 'P': 3.159,
    'Q': 1.290, 'S': 1.444,
    'E': 0.928, 'T': 1.172,
    'G': 1.015, 'W': 0.452,
    'H': 0.611, 'Y': 0.816,
    'I': 0.603, 'V': 0.640
}
CEDJ970105={
    'A': 8.3, 'L': 7.4,
    'R': 8.7, 'K': 7.9,
    'N': 3.7, 'M': 2.3,
    'D': 4.7, 'F': 2.7,
    'C': 1.6, 'P': 6.9,
    'Q': 4.7, 'S': 8.8,
    'E': 6.5, 'T': 5.1,
    'G': 6.3, 'W': 0.7,
    'H': 2.1, 'Y': 2.4,
    'I': 3.7, 'V': 5.3
}
QIAN880127={
    'A': -0.05, 'L': 0.04,
    'R': 0.06, 'K': -0.42,
    'N': 0.00, 'M': 0.25,
    'D': 0.15, 'F': 0.09,
    'C': 0.30, 'P': 0.31,
    'Q': -0.08, 'S': -0.11,
    'E': -0.02, 'T': -0.06,
    'G': -0.14, 'W': 0.19,
    'H': -0.07, 'Y': 0.33,
    'I': 0.26, 'V': 0.04
}
LEVM760107={
    'A': 0.025, 'L': 0.19,
    'R': 0.20, 'K': 0.19,
    'N': 0.10, 'M': 0.19,
    'D': 0.10, 'F': 0.39,
    'C': 0.10, 'P': 0.17,
    'Q': 0.10, 'S': 0.025,
    'E': 0.10, 'T': 0.10,
    'G': 0.025, 'W': 0.56,
    'H': 0.10, 'Y': 0.39,
    'I': 0.19, 'V': 0.15
}
def autocorrelation(protein_list, lag, AAindex_list):
    autocorrelation = []
    for sequence in protein_list:
        temp=[]
        for property_values in AAindex_list:
        # 将氨基酸序列转换为属性值序列
        #     property_values = np.array([property_dict[aa] for aa in sequence])
            property_values = [0 if value is None else value for value in property_values]
            # 计算属性值的平均值
            # print(property_values)
            mean_value = np.mean(property_values)
            # print(mean_value)
            # 计算Moran自相关
            n = len(sequence)
            autocorr = np.sum((property_values[:-lag] - mean_value) * (property_values[lag:] - mean_value))
            autocorr /= (n - lag)
            temp.append(autocorr)
            
        autocorrelation.append(temp)
    
    return autocorrelation

hydrophobicity = {
    'A': 1.8, 'C': 2.5, 'D': -3.5, 'E': -3.5, 'F': 2.8, 'G': -0.4, 'H': -3.2, 'I': 4.5, 
    'K': -3.9, 'L': 3.8, 'M': 1.9, 'N': -3.5, 'P': -1.6, 'Q': -3.5, 'R': -4.5, 'S': -0.8,
    'T': -0.7, 'V': 4.2, 'W': -0.9, 'Y': -1.3
}

lag = 1  # 步长
import pandas as pd
    # """加载AAindex数据，假设第一列为描述，其余列为各个氨基酸的物理化学属性值。"""
import csv

# Path to the CSV file
csv_file_path = 'aaindex1.csv'

# Initialize an empty list to store the data
data = []

# Open the CSV file and read it
with open(csv_file_path, mode='r', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    
    # Skip the first cell of the first row
    for row in csv_reader:
        # Append the row with the first cell removed to the data list
        # data.append([float(value) for value in row[1:]])
        data.append(row[1:])
data=data[1:]
# float_data = [[float(value) for value in row] for row in data]


AAindex_data = [
    [float(value) if value not in ['NA', ''] else None for value in row]
    for row in data
]

# AAindex_list=[ NAKH900113, KRIW710101, HUTJ700103,ZIMJ680103,TANS770104,CEDJ970105, QIAN880127, LEVM760107]

autocorrelation_train = autocorrelation(sequences, lag, AAindex_data)
autocorrelation_test = autocorrelation(sequences_test, lag, AAindex_data)
len(autocorrelation_train),len(autocorrelation_train[0])

(951, 566)

In [8]:
import scipy.stats.stats as st
def aaindex(path):
    protein_list = []
    csv_file_path = 'aaindex1.csv'
    lag=2
    data = []
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        csv_reader = csv.reader(file)
        
        # Skip the first cell of the first row
        for row in csv_reader:
            # Append the row with the first cell removed to the data list
            # data.append([float(value) for value in row[1:]])
            data.append(row[1:])
    data=data[1:]
    AAindex_list = [
        [float(value) if value not in ['NA', ''] else None for value in row]
        for row in data
    ]
    # NAKH900113 【200】，KRIW710101【146】， HUTJ700103【117】，ZIMJ680103【399】，TANS770104【368】，CEDJ970105【459】，QIAN880127【283】，LEVM760107【158】
    # selected_indices = [200, 146, 117, 399, 368,459,283, 158]
    # AAindex_list = [AAindex_list[i] for i in selected_indices if i < len(AAindex_list)]
    with open(path, 'r') as file:
        for line in file:
            line = line.strip()
            if not line.startswith(">"):  # 忽略ID行，只保存序列行
                protein_list.append(line)
                
    autocorrelation = []
    for sequence in protein_list:
        temp=[]
        for property_values in AAindex_list:
        # 将氨基酸序列转换为属性值序列
        #     property_values = np.array([property_dict[aa] for aa in sequence])
            property_values = [0 if value is None else value for value in property_values]
            # 计算属性值的平均值
            # print(property_values)
            mean_value = np.mean(property_values)
            # print(mean_value)
            # 计算Moran自相关
            n = len(sequence)
            autocorr = np.sum((property_values[:-lag] - mean_value) * (property_values[lag:] - mean_value))
            autocorr /= (n - lag)
            temp.append(autocorr)

        autocorrelation.append(temp)
        
    v = []
    for i in range(len(autocorrelation)):
        vtar = autocorrelation[i]
        vtarv = []
        vtar7 = 0
        vtar8 = 0
        vtar9 = 0
        s = pd.Series(vtar)
        vtar3 = np.mean(vtar)  # These 4 dimensions are relevant statistical terms
        vtar4 = st.kurtosis(vtar)
        vtar5 = np.var(vtar)
        vtar6 = st.skew(vtar)
        #for p in range(len(vtar)): # These 3 dimensions are inspired by PAFIG algorithm
        #vtar7=vtar[p]**2+vtar7
        #if vtar[p]>va:
        #vtar8=vtar[p]**2+vtar8
        #else:
        #vtar9=vtar[p]**2+vtar9
        vcf1 = []
        vcf2 = []
        for j in range(len(vtar) - 1):  #Sequence-order-correlation terms
            vcf1.append((vtar[j] - vtar[j + 1]))
        for k in range(len(vtar) - 2):
            vcf2.append((vtar[k] - vtar[k + 2]))
        vtar10 = np.mean(vcf1)
        vtar11 = np.var(vcf1)
        vtar11A = st.kurtosis(vcf1)
        vtar11B = st.skew(vcf1)
        vtar12 = np.mean(vcf2)
        vtar13 = np.var(vcf2)
        vtar13A = st.kurtosis(vcf2)
        vtar13B = st.skew(vcf2)
        vtarv.append(vtar3)
        vtarv.append(vtar4)
        vtarv.append(vtar5)
        vtarv.append(vtar6)
        #vtarv.append(vtar7/len(vtar))
        #vtarv.append(vtar8/len(vtar))
        #vtarv.append(vtar9/len(vtar))
        vtarv.append(vtar10)
        vtarv.append(vtar11)
        vtarv.append(vtar11A)
        vtarv.append(vtar11B)
        vtarv.append(vtar12)
        vtarv.append(vtar13)
        vtarv.append(vtar13A)
        vtarv.append(vtar13B)
        v.append(vtarv)
    return v

train_path = '/tmp/pycharm_project_763/raw/AVP.txt'
test_path ='/tmp/pycharm_project_763/raw/AVPT.txt'
v_train=aaindex(train_path)
v_test=aaindex(test_path)
v_train = np.array(v_train)  # 转换为 NumPy 数组
v_test = np.array(v_test)
# print(v.shape,type(v))
# 检查是否为二维数组


v_train = (v_train - np.mean(v_train, axis=1, keepdims=True)) / np.std(v_train, axis=1, keepdims=True)
v_test = (v_test - np.mean(v_test, axis=1, keepdims=True)) / np.std(v_test, axis=1, keepdims=True)
v_train.shape,v_test.shape

((951, 12), (105, 12))

In [9]:
def EGAAC(sequences, window=5):
    if window < 1:
        print('Error: the sliding window should be greater than zero' + '\n\n')
        return None

    group = {
        'alphatic': 'GAVLMI',
        'aromatic': 'FYW',
        'positive_charge': 'KRH',
        'negative_charge': 'DE',
        'uncharged': 'STCPNQ'
    }

    groupKeys = group.keys()

    encodings = []
    header = ['#']
    max_len = max(len(seq) for seq in sequences)  # Find the maximum length of sequences
    for w in range(1, max_len - window + 2):
        for g in groupKeys:
            header.append('SW.' + str(w) + '.' + g)
    # encodings.append(header)

    for sequence in sequences:
        code = []
        for j in range(len(sequence) - window + 1):
            subseq = sequence[j:j + window]
            count = Counter(subseq)
            myDict = {}
            for key in groupKeys:
                myDict[key] = sum(count[aa] for aa in group[key] if aa in count)
            for key in groupKeys:
                code.append(myDict[key] / window)
        encodings.append(code)

    return encodings

# 示例蛋白质序列列表

EGAAC_train = EGAAC(sequences, window=5)
EGAAC_test = EGAAC(sequences_test, window=5)
max_length1 = max(len(item) for item in EGAAC_train)
max_length2 = max(len(item) for item in EGAAC_test)
max_length=max(max_length1,max_length2)

# 进行零填充
EGAAC_train = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in EGAAC_train])
EGAAC_test = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in EGAAC_test])
EGAAC_train.shape

(951, 515)

In [10]:
from Bio.SeqUtils.ProtParam import ProteinAnalysis
def calculate_properties(protein_sequences):
    results = []
    for sequence in protein_sequences:
        
        analyzed_seq = ProteinAnalysis(sequence)
        properties = {
            "Molecular Weight": analyzed_seq.molecular_weight(),
            "Aromaticity": analyzed_seq.aromaticity(),
            "Instability Index": analyzed_seq.instability_index(),
            "Isoelectric Point": analyzed_seq.isoelectric_point(),
            "Secondary Structure Fraction": analyzed_seq.secondary_structure_fraction(),  # Returns tuple (helix, turn, sheet)
            "Gravy": analyzed_seq.gravy()  # Grand average of hydropathicity
        }
        results.append([analyzed_seq.molecular_weight(),analyzed_seq.aromaticity(),analyzed_seq.instability_index(),analyzed_seq.instability_index(),analyzed_seq.isoelectric_point(),analyzed_seq.secondary_structure_fraction()[0],analyzed_seq.secondary_structure_fraction()[1],analyzed_seq.secondary_structure_fraction()[2],analyzed_seq.gravy() ])
        # results.append(properties)
    return results
properties_train=calculate_properties(sequences)
properties_test=calculate_properties(sequences_test)
#长度9

In [11]:
def encode_proteins_with_blosum62(protein_list, blosum62):
    encoded_proteins = []
    for protein in protein_list:
        protein_encoding = []
        for amino_acid in protein:
            if amino_acid in blosum62:
                # 将氨基酸的BLOSUM62编码添加到当前蛋白质的编码列表中
                protein_encoding.extend(blosum62[amino_acid])
            else:
                # 对于不在BLOSUM62中的氨基酸，可以添加一个全0向量
                protein_encoding.extend([0] * len(blosum62['A']))
        encoded_proteins.append(protein_encoding)
    return encoded_proteins

# BLOSUM62矩阵定义
blosum62 = {
		'A': [4,  -1, -2, -2, 0,  -1, -1, 0, -2,  -1, -1, -1, -1, -2, -1, 1,  0,  -3, -2, 0],  # A
		'R': [-1, 5,  0,  -2, -3, 1,  0,  -2, 0,  -3, -2, 2,  -1, -3, -2, -1, -1, -3, -2, -3], # R
		'N': [-2, 0,  6,  1,  -3, 0,  0,  0,  1,  -3, -3, 0,  -2, -3, -2, 1,  0,  -4, -2, -3], # N
		'D': [-2, -2, 1,  6,  -3, 0,  2,  -1, -1, -3, -4, -1, -3, -3, -1, 0,  -1, -4, -3, -3], # D
		'C': [0,  -3, -3, -3, 9,  -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1], # C
		'Q': [-1, 1,  0,  0,  -3, 5,  2,  -2, 0,  -3, -2, 1,  0,  -3, -1, 0,  -1, -2, -1, -2], # Q
		'E': [-1, 0,  0,  2,  -4, 2,  5,  -2, 0,  -3, -3, 1,  -2, -3, -1, 0,  -1, -3, -2, -2], # E
		'G': [0,  -2, 0,  -1, -3, -2, -2, 6,  -2, -4, -4, -2, -3, -3, -2, 0,  -2, -2, -3, -3], # G
		'H': [-2, 0,  1,  -1, -3, 0,  0,  -2, 8,  -3, -3, -1, -2, -1, -2, -1, -2, -2, 2,  -3], # H
		'I': [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4,  2,  -3, 1,  0,  -3, -2, -1, -3, -1, 3],  # I
		'L': [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2,  4,  -2, 2,  0,  -3, -2, -1, -2, -1, 1],  # L
		'K': [-1, 2,  0,  -1, -3, 1,  1,  -2, -1, -3, -2, 5,  -1, -3, -1, 0,  -1, -3, -2, -2], # K
		'M': [-1, -1, -2, -3, -1, 0,  -2, -3, -2, 1,  2,  -1, 5,  0,  -2, -1, -1, -1, -1, 1],  # M
		'F': [-2, -3, -3, -3, -2, -3, -3, -3, -1, 0,  0,  -3, 0,  6,  -4, -2, -2, 1,  3,  -1], # F
		'P': [-1, -2, -2, -1, -3, -1, -1, -2, -2, -3, -3, -1, -2, -4, 7,  -1, -1, -4, -3, -2], # P
		'S': [1,  -1, 1,  0,  -1, 0,  0,  0,  -1, -2, -2, 0,  -1, -2, -1, 4,  1,  -3, -2, -2], # S
		'T': [0,  -1, 0,  -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 1,  5,  -2, -2, 0],  # T
		'W': [-3, -3, -4, -4, -2, -2, -3, -2, -2, -3, -2, -3, -1, 1,  -4, -3, -2, 11, 2,  -3], # W
		'Y': [-2, -2, -2, -3, -2, -1, -2, -3, 2,  -1, -1, -2, -1, 3,  -3, -2, -2, 2,  7,  -1], # Y
		'V': [0,  -3, -3, -3, -1, -2, -2, -3, -3, 3,  1,  -2, 1,  -1, -2, -2, 0,  -3, -1, 4],  # V
		'-': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # -
	}

# 示例使用

blosum62_train = encode_proteins_with_blosum62(sequences, blosum62)
blosum62_test = encode_proteins_with_blosum62(sequences_test, blosum62)

max_length1 = max(len(item) for item in blosum62_train)
max_length2 = max(len(item) for item in blosum62_test)
max_length=max(max_length1,max_length2)

# 进行零填充
blosum62_train = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in blosum62_train])
blosum62_test = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in blosum62_test])
blosum62_test.shape,blosum62_train.shape

((105, 2140), (951, 2140))

In [12]:
def one_hot_encode_proteins(protein_list):
    # 定义氨基酸的独热编码字典
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'  # 常见的20种氨基酸
    aa_to_onehot = {aa: np.eye(len(amino_acids))[i] for i, aa in enumerate(amino_acids)}

    one_hot_encoded = []

    for protein in protein_list:
        # 将蛋白质序列转换为独热编码
        encoded = [aa_to_onehot[aa] for aa in protein if aa in aa_to_onehot]
        
        # 将编码展平成一维向量
        encoded_flattened = np.array(encoded).flatten()
        
        
        one_hot_encoded.append(encoded_flattened)

    return one_hot_encoded

# 示例使用

one_hot_train = one_hot_encode_proteins(sequences)
one_hot_test = one_hot_encode_proteins(sequences_test)

max_length1 = max(len(item) for item in one_hot_train)
max_length2 = max(len(item) for item in one_hot_test)
max_length=max(max_length1,max_length2)

# 进行零填充
one_hot_train = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in one_hot_train])
one_hot_test = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in one_hot_test])
one_hot_train.shape,one_hot_test.shape


((951, 2140), (105, 2140))

In [13]:
from collections import Counter

def calculate_dpc(protein_list):
    # 定义20种常见氨基酸
    amino_acids = 'ACDEFGHIKLMNPQRSTVWY'
    
    # 生成所有可能的二肽组合
    dipeptides = [aa1 + aa2 for aa1 in amino_acids for aa2 in amino_acids]
    
    dpc_vectors = []
    
    for sequence in protein_list:
        # 统计每个序列中的二肽频率
        counts = Counter([sequence[i:i+2] for i in range(len(sequence) - 1)])
        
        # 计算归一化的DPC特征
        dpc_vector = np.array([counts[dipeptide] for dipeptide in dipeptides], dtype=float)
        dpc_vector /= (len(sequence) - 1)  # 归一化
        
        dpc_vectors.append(dpc_vector)
    
    return np.array(dpc_vectors)
dpc_train=calculate_dpc(sequences)
dpc_test=calculate_dpc(sequences_test)
dpc_train.shape

(951, 400)

In [14]:
import pandas as pd
import numpy as np

def load_aaindex(filepath):
    """加载AAindex数据，假设第一列为描述，其余列为各个氨基酸的物理化学属性值。"""
    aaindex_data = pd.read_csv(filepath, index_col=0)
    return aaindex_data

def encode_protein_AAindex(protein_sequences, aaindex_data):
    """根据AAindex编码蛋白质序列为一维特征向量。
    
    参数:
        protein_sequences (list of str): 蛋白质序列列表。
        aaindex_data (DataFrame): 从CSV文件加载的AAindex数据。
    
    返回:
        list of numpy.array: 编码后的蛋白质序列特征向量列表。
    """
    encoded_features = []
    for sequence in protein_sequences:
        # 创建一个空数组来存储每个氨基酸的特征
        features = []
        for amino_acid in sequence:
            if amino_acid in aaindex_data.columns:
                # 添加当前氨基酸的所有属性到特征列表
                features.append(aaindex_data[amino_acid].values)
            else:
                # 对于不存在的氨基酸，添加一个全0向量
                features.append(np.zeros(aaindex_data.shape[0]))
        # 将特征列表转换为二维数组，然后展平为一维
        flat_features = np.concatenate(features)
        encoded_features.append(flat_features)
    return encoded_features

# Example usage
filepath = 'aaindex2.csv'  # AAindex文件路径
aaindex = load_aaindex(filepath)  # 加载AAindex数据

  # 示例蛋白质序列
AAindex_train = encode_protein_AAindex(sequences, aaindex)
AAindex_test = encode_protein_AAindex(sequences_test, aaindex)

max_length1 = max(len(item) for item in AAindex_train)
max_length2 = max(len(item) for item in AAindex_test)
max_length=max(max_length1,max_length2)

# 进行零填充
AAindex_train = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in AAindex_train])
AAindex_test = np.array([np.pad(item, (0, max_length - len(item)), 'constant', constant_values=(0)) for item in AAindex_test])
AAindex_test.shape,AAindex_train.shape

((105, 221276), (951, 221276))

In [15]:
len(aac_train),len(aac_train[0]),len(maccs_train),len(maccs_train[0]),len(v_train),len(v_test)

(951, 20, 951, 167, 951, 105)

In [38]:
# from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler
# min_max_scaler = MinMaxScaler()
# properties_train = min_max_scaler.fit_transform(properties_train)
# properties_test = min_max_scaler.fit_transform(properties_test)

# train_encodings=np.concatenate((aac_train,maccs_train,v_train),axis=1)
# test_encodings=np.concatenate((aac_test,maccs_test,v_test),axis=1)
train_encodings=np.array(maccs_train)
test_encodings=np.array(maccs_test)
# train_encodings=autocorrelation_train
# test_encodings=autocorrelation_test

train_encodings.shape,test_encodings.shape,len(y_test)
# one_hot_train[0],properties_train[0],aac_train[0],EGAAC_train[0],AAindex_train[0],blosum62_train[0],dpc_train[0],
# train_encodings[2],EGAAC_train[0],properties_train[0],len(EGAAC_train[0])

((951, 167), (105, 167), 105)

In [39]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(train_encodings, y, random_state=42)
X_test, y_test = shuffle(test_encodings, y_test, random_state=42)
X_train=np.array(X_train)
X_test=np.array(X_test)
y_train=np.array(y_train)
y_test=np.array(y_test)
X_train = torch.tensor(X_train)
y_train = torch.tensor(y_train).view(-1, 1)
X_test = torch.tensor(X_test)
y_test = torch.tensor(y_test).view(-1, 1)
X_train = torch.tensor(X_train ,dtype=torch.float32)
y_train = torch.tensor(y_train,dtype=torch.float32)
X_test = torch.tensor(X_test,dtype=torch.float32)
y_test = torch.tensor(y_test,dtype=torch.float32)

/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  if sys.path[0] == "":
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  del sys.path[0]
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:14: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  
/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/ipykernel_launcher.py:15: UserWarning: To

In [40]:
dataset = {}
dataset = {
    'train_input': X_train,
    'test_input': X_test,
    'train_label': y_train,
    'test_label': y_test
}
X_train.shape

torch.Size([951, 167])

In [41]:
y_test.shape

torch.Size([105, 1])

In [42]:
import numpy as np
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

model = SVC(kernel='linear', probability=True)  # probability=True为了后续计算AUC
X_train,X_test=X_test,X_train
y_train,y_test=y_test,y_train
# 训练模型
model.fit(X_train, y_train)

# 预测
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 计算准确度
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 计算AUC
y_train_prob = model.predict_proba(X_train)[:, 1]
y_test_prob = model.predict_proba(X_test)[:, 1]
train_auc = roc_auc_score(y_train, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# 输出结果
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Training AUC: {train_auc:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Test AUC: {test_auc:.2f}")

Training Accuracy: 0.78
Training AUC: 0.86
Test Accuracy: 0.63
Test AUC: 0.67


/home/wangli/anaconda3/envs/mykan/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [51]:
model = KAN(width=[167,300,300,300,1], grid=5, k=3, seed=42)

checkpoint directory created: ./model
saving model version 0.0


In [52]:
from sklearn.metrics import roc_auc_score, confusion_matrix, matthews_corrcoef
def train_acc():
    pred=model(X_train)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_train.cpu(), pred.cpu())

    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_train.cpu(), pred_labels.cpu()).ravel()

        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        acc = (tp + tn) / (tp + tn + fp + fn)

        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0

        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_train.cpu(), pred_labels.cpu())
    print(f"ACC: {acc:.4f}, AUC: {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_train)[:, 0]) == y_train[:, 0]).float())

def test_acc():
    pred=model(X_test)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_test.cpu(), pred.cpu())

    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_test.cpu(), pred_labels.cpu()).ravel()

        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0
        
        acc = (tp + tn) / (tp + tn + fp + fn)

        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0

        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_test.cpu(), pred_labels.cpu())
    print(f"ACC: {acc:.4f}, AUC(test): {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_test)[:, 0]) == y_test[:, 0]).float())

# results = model.train(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc)) ,lamb=0.001 lamb_entropy=4.,lamb=0.1,lamb_l1=2.5,
# lamb=0.005 train/fit
results = model.fit(dataset, opt="LBFGS", steps=30, metrics=(train_acc, test_acc));
# print(results['train_acc'][-1], results['test_acc'][-1])

description:   0%|                                                           | 0/30 [00:00<?, ?it/s]

ACC: 0.6000, AUC: 0.6230, SP: 0.7500, MCC: 0.1599, SN: 0.4000
ACC: 0.6604, AUC(test): 0.7113, SP: 0.8125, MCC: 0.2902, SN: 0.4570


| train_loss: 4.61e-01 | test_loss: 4.96e-01 | reg: 0.00e+00 | :   3%| | 1/30 [00:46<22:18, 46.16s/i

ACC: 0.6762, AUC: 0.7807, SP: 0.8000, MCC: 0.3266, SN: 0.5111
ACC: 0.7508, AUC(test): 0.8362, SP: 0.8474, MCC: 0.4855, SN: 0.6216


| train_loss: 4.10e-01 | test_loss: 4.46e-01 | reg: 0.00e+00 | :   7%| | 2/30 [01:21<18:31, 39.70s/i

ACC: 0.6952, AUC: 0.7707, SP: 0.7500, MCC: 0.3746, SN: 0.6222
ACC: 0.8339, AUC(test): 0.9201, SP: 0.8768, MCC: 0.6590, SN: 0.7764


| train_loss: 3.46e-01 | test_loss: 4.65e-01 | reg: 0.00e+00 | :  10%| | 3/30 [02:00<17:40, 39.27s/i

ACC: 0.6762, AUC: 0.7615, SP: 0.8000, MCC: 0.3266, SN: 0.5111
ACC: 0.8801, AUC(test): 0.9527, SP: 0.9081, MCC: 0.7545, SN: 0.8428


| train_loss: 3.09e-01 | test_loss: 4.70e-01 | reg: 0.00e+00 | :  13%|▏| 4/30 [02:53<18:46, 43.34s/i


KeyboardInterrupt: 

In [172]:
model = model.refine(10)

IndexError: list index out of range

In [ ]:
from sklearn.metrics import roc_auc_score, confusion_matrix, matthews_corrcoef
def train_acc():
    pred=model(X_train)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_train.cpu(), pred.cpu())

    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_train.cpu(), pred_labels.cpu()).ravel()

        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0

        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0

        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_train.cpu(), pred_labels.cpu())
    print(f"AUC: {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_train)[:, 0]) == y_train[:, 0]).float())

def test_acc():
    pred=model(X_test)
    with torch.no_grad():
        pred_labels = (pred > 0.5).float() 
        auc = roc_auc_score(y_test.cpu(), pred.cpu())

    # 混淆矩阵计算
        tn, fp, fn, tp = confusion_matrix(y_test.cpu(), pred_labels.cpu()).ravel()

        # Specificity (SP)
        sp = tn / (tn + fp) if (tn + fp) > 0 else 0

        # Sensitivity (SN)
        sn = tp / (tp + fn) if (tp + fn) > 0 else 0

        # Matthews Correlation Coefficient (MCC)
        mcc = matthews_corrcoef(y_test.cpu(), pred_labels.cpu())
    print(f"AUC(test): {auc:.4f}, SP: {sp:.4f}, MCC: {mcc:.4f}, SN: {sn:.4f}")
    return torch.mean((torch.round(model(X_test)[:, 0]) == y_test[:, 0]).float())

# results = model.train(dataset, opt="LBFGS", steps=10, metrics=(train_acc, test_acc)) ,lamb=0.001 lamb_entropy=4.,lamb=0.1,lamb_l1=2.5,
# lamb=0.005 train/fit
results = model.fit(dataset, opt="LBFGS", steps=5, metrics=(train_acc, test_acc));
print(results['train_acc'][-1], results['test_acc'][-1])

checkpoint directory created: ./model
saving model version 0.0


In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
param_grid = {
    'hidden_layer_sizes': [(50,), (100,), (50,50)],
    'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],
    'alpha': [0.0001, 0.05],
    'learning_rate': ['constant','adaptive'],
}
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=300, alpha=1e-4,
                    solver='sgd', verbose=10, random_state=1,
                    learning_rate_init=.1)
mlp.fit(X_train, y_train)
predictions_train = mlp.predict(X_train)
predictions_test = mlp.predict(X_test)

# 计算准确率
train_accuracy = accuracy_score(y_train, predictions_train)
test_accuracy = accuracy_score(y_test, predictions_test)

print(f'Train Accuracy: {train_accuracy:.4f}')
print(f'Test Accuracy: {test_accuracy:.4f}')

description:   0%|                                                           | 0/30 [00:00<?, ?it/s]

In [27]:
import numpy as np
from sklearn import datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

model = SVC(kernel='linear', probability=True)  # probability=True为了后续计算AUC
X_train,X_test=X_test,X_train
y_train,y_test=y_test,y_train
# 训练模型
model.fit(X_train, y_train)

# 预测
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# 计算准确度
train_accuracy = accuracy_score(y_train, y_train_pred)
test_accuracy = accuracy_score(y_test, y_test_pred)

# 计算AUC
y_train_prob = model.predict_proba(X_train)[:, 1]
y_test_prob = model.predict_proba(X_test)[:, 1]
train_auc = roc_auc_score(y_train, y_train_prob)
test_auc = roc_auc_score(y_test, y_test_prob)

# 输出结果
print(f"Training Accuracy: {train_accuracy:.2f}")
print(f"Test Accuracy: {test_accuracy:.2f}")
print(f"Training AUC: {train_auc:.2f}")
print(f"Test AUC: {test_auc:.2f}")

/home/wangli/anaconda3/envs/transformer-env/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Training Accuracy: 0.59
Test Accuracy: 0.66
Training AUC: 0.60
Test AUC: 0.70


description:   0%|                                                           | 0/30 [11:04<?, ?it/s]


KeyboardInterrupt: 